In [29]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit_aer.primitives import Estimator
import matplotlib.pyplot as plt
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2, EstimatorOptions
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

In [ ]:
# setting up the service
service = QiskitRuntimeService(channel='ibm_quantum',
                               token='87f82131330e8f3630c9998b85e000569b57a405e9d152e188000d3864901c8ee6106712bdb2410b20cf366224821e6e9a5c622249b2b9da0ffc2c3342cfcfa8')

## $n$-qubit GHZ state

Let's extend the Hello World 1-qubit example to something more realistic, an $n$-qubit GHZ state. This is $\ket{\psi}=A\left(\ket{0\cdots 0} + \ket{1\cdots 1}\right)$ where $A$ is the normalization

In [14]:
# contruct a function to make the process more flexible
def qc_n_ghz_state(n):
    qc = QuantumCircuit(n)
    # apply hadamard gate
    qc.h(0)
    # apply ladder of CNOTs
    for i in range(n-1):
        qc.cx(control_qubit=i, target_qubit=i+1)

    return qc

# example
n = 100
qc = qc_n_ghz_state(n)
#qc.draw(output='mpl')

In [15]:
# see the expectation value of ZZ for any pair of qubits to see the errors
operator_strings = ['Z'+'I'*i+'Z'+'I'*(n-2-i) for i in range(n-1)]
#print(operator_strings)
print(len(operator_strings))

# create operators
operators = [SparsePauliOp(operator_string) for operator_string in operator_strings]

99


In [46]:
# now we want to optimize for quantum execution
#backend_name = 'ibm_brisbane'
#backend = service.backend(name=backend_name)
backend = service.least_busy(min_num_qubits=100, operational=True)
pass_manager = generate_preset_pass_manager(optimization_level=1, backend=backend)

# transpile
qc_transpiled = pass_manager.run(qc)
operators_transpiled = [op.apply_layout(qc_transpiled.layout) for op in operators]

In [ ]:
# # execute on the backend
# options = EstimatorOptions()
# options.resilience_level = 1
# #options.optimization_level = 0
# options.dynamical_decoupling.enable = True
# options.dynamical_decoupling.sequence_type = 'XY4'

# estimator = EstimatorV2(backend, options=options)

# # run
# job = estimator.run([(qc_transpiled, operators_transpiled)])
# # id
# job_id = job.job_id()
# print(job_id)

czp6k0wd8drg008gantg


In [ ]:
# post processing
job_id = 'czp6k0wd8drg008gantg'
job = service.job(job_id)

data = list(range(1, len(operators)+1))
result = job.result()[0]
values = result.data.evs
values = [v/values[0] for v in values]

# plot
fig, ax = plt.subplots()

ax.scatter(data, values, marker='-o', label='100-qubit GHZ state')
ax.set_xlabel('Distance between qubits $i$')
ax.set_ylabel('$\\langle Z_0Z_i \\rangle \\langle Z_0Z_1 \\rangle$')
ax.grid()
ax.legend()